In [2]:
from mypulp import * 
import numpy as np
import itertools

# 箱詰め問題の定式化

例題

In [18]:
def create_example():
    B = 9
    w = [2, 3, 4, 5, 6, 7, 8]
    q = [4, 2, 6, 6, 2, 2, 2]
    s = []
    
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    
    return s, B

ビンの数の上限を計算 -> サイズの大きいアイテムを順番に入れていく方法（First Fit Decreasing; FFD）を用いて計算

In [31]:
def FFD(s, B):
    remain = [B]  # 各ビンの空き容量
    sol = [[]]  # 解を保管するリスト
    items = np.sort(s)[::-1]
    
    for item in items:
        # もし入れられる余白のあるビンがあればそこに入れる
        for j, rest in enumerate(remain):
            if rest >= item:
                remain[j] = remain[j] - item
                sol[j].append(item)
                success = True
                break
            # 入れられるビンがなければビンを追加してそこに入れる
            success = False
        if not success:
            sol.append([item])
            remain.append(B-item)
    
    return sol

In [60]:
def solve_bpp(s, B):
    model = Model('bpp')
    
    ## 変数 ##
    x, y = {}, {}
    for j in J:
        y[j] = model.addVar(vtype='B', name=f'y_{j}')
        for i in I:
            x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
            
    model.update()
    
    ## 定式化 ##
    for i in I:
        model.addConstr(quicksum(x[i, j] for j in J) == 1)
    
    for j in J:
        model.addConstr(quicksum(s[i] * x[i, j] for i in I) <= B)
    
    for i, j in itertools.product(I, J):
        model.addConstr(x[i, j] <= y[j])
    
    model.setObjective(quicksum(y[j] for j in J), GRB.MINIMIZE)
    
    model.update()
    model.__data = x, y
    
    return model    

In [62]:
s, B = create_example()
I = range(len(s))
J = range(len(FFD(s, B)))

model = solve_bpp(s, B)
model.optimize()
x, y = model.__data

# 最適値
print(f'Optimal Value: {model.ObjVal}')

Optimal Value: 13.0
